In [4]:
from azure.identity import ClientSecretCredential
from azure.mgmt.containerinstance import ContainerInstanceManagementClient
from azure.mgmt.containerinstance.models import (
    ContainerGroup, Container, ResourceRequests, ResourceRequirements, 
    ContainerGroupNetworkProtocol, OperatingSystemTypes, 
    IpAddress, Port
)
from dotenv import load_dotenv
import os
import time

# Load environment variables from .env file
load_dotenv()

# Get values from .env file
SUBSCRIPTION_ID = os.getenv('SUBSCRIPTION_ID')
RESOURCE_GROUP = os.getenv('RESOURCE_GROUP')
CONTAINER_NAME = 'aci-app'  # Name of the container group
IMAGE = os.getenv('IMAGE', 'sk4123/aci-app:latest')  # Docker Hub image you pushed earlier
CPU_CORE_COUNT = float(os.getenv('CPU_CORE_COUNT', 2.0))
MEMORY_GB = float(os.getenv('MEMORY_GB', 4))
TENANT_ID = os.getenv('TENANT_ID')
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

# Get credentials
credentials = ClientSecretCredential(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    tenant_id=TENANT_ID
)

# Create a Container Instance Management client
container_client = ContainerInstanceManagementClient(credentials, SUBSCRIPTION_ID)

# Define the container
container_resource_requests = ResourceRequests(memory_in_gb=MEMORY_GB, cpu=CPU_CORE_COUNT)
container_resource_requirements = ResourceRequirements(requests=container_resource_requests)
container = Container(
    name=CONTAINER_NAME, 
    image=IMAGE, 
    resources=container_resource_requirements, 
    ports=[Port(port=8000)]  # Ensure this matches the exposed port in your Dockerfile
)

# Define the group of containers
container_group = ContainerGroup(
    location='westeurope',  # Adjust location as needed
    containers=[container], 
    os_type=OperatingSystemTypes.linux, 
    ip_address=IpAddress(
        ports=[Port(protocol=ContainerGroupNetworkProtocol.tcp, port=8000)], 
        type='Public'
    )
)

# Create the container group
deployment_result = container_client.container_groups.begin_create_or_update(RESOURCE_GROUP, CONTAINER_NAME, container_group).result()

# Wait for a moment to ensure the deployment is completed and the IP address is assigned
time.sleep(60)

# Retrieve the container group details
container_group_details = container_client.container_groups.get(RESOURCE_GROUP, CONTAINER_NAME)

# Get the IP address
ip_address = container_group_details.ip_address.ip

print(f"The IP address of the deployed container is: {ip_address}")


The IP address of the deployed container is: 98.64.96.137


In [2]:
# Get the location
location = container_group_details.location
print(f"The location of the deployed container is: {location}")

The location of the deployed container is: westeurope


In [3]:
fqdn = container_group_details.ip_address.fqdn
print(f"The URL of the deployed container is: {fqdn}")

The URL of the deployed container is: None


In [9]:
import requests

# The URL of the API
container_IP = '20.50.226.153'
api_url = f'http://{container_IP}:8000/'
endpoint = 'predict/'
url = api_url + endpoint

# Load the image file
image_path = 'C:/Users/stijn/Documents/2023-24d-fai2-adsai-group-cv-3/data/testing_dataset/test/images/images/002_43-19-ROOT1-2023-08-08_pvd_OD001_Col0_01-Fish Eye Corrected_0.png'
image_file = open(image_path, 'rb')

# Create the payload
files = {'file': image_file}

# Send the POST request
response = requests.post(url, files=files)

# Print the response status code and raw content for debugging
print(f"Status Code: {response.status_code}")
print(f"Response Content: {response.text}")

# Attempt to print the JSON response if possible
try:
    print(response.json())
except requests.JSONDecodeError:
    print("Response is not in JSON format.")


Status Code: 200
Response Content: {"prediction":[[[[5.980552231221736e-08],[2.999222536992363e-10],[3.437956586083146e-10],[4.75349230463773e-12],[1.3811533568306583e-13],[1.9926074001528835e-14],[7.956118945086355e-13],[6.983702624443339e-14],[9.172180972637917e-15],[2.983046328805831e-15],[3.1537944998605294e-13],[3.854766305417742e-14],[8.942975551176797e-15],[3.780343618979727e-15],[4.32809305972795e-13],[5.468131644096458e-14],[1.3602975591374324e-14],[6.132148384723184e-15],[5.624842497158056e-13],[8.033241498221766e-14],[2.2348347673319113e-14],[9.693889740675522e-15],[6.733133473511843e-13],[8.899436363622032e-14],[2.488463536794161e-14],[1.0392054106351037e-14],[6.958535852565062e-13],[9.126167432437632e-14],[2.5494539747546174e-14],[1.0571542081940118e-14],[7.046471696067758e-13],[9.238042188858264e-14],[2.580450976052799e-14],[1.0700808627053544e-14],[7.140598333975345e-13],[9.34811445916714e-14],[2.60924552228153e-14],[1.0839881272597333e-14],[7.216340697745183e-13],[9.535